# Fitting

There are several ways (e.g. using [least square fit](https://en.wikipedia.org/wiki/Least_squares) or the [Likelihood function](https://en.wikipedia.org/wiki/Likelihood_function)) to fit a model (a simple line, a polynomial of degree n or machine learning model) to your data. In general, we call this step **fitting** or **regression** and it is always a kind of minimization task. In Python there are several modules for that purpose (see also [SciPy optimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) or [Numpy polyfit](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html)).

Parameter selection is a crucial step - be it choosing the appropriate degree of freedom of a polynomial or the appropriate set of parameters for a more complex machine learning algorithm.

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.optimize as opt
import scipy.stats as stats


plt.rcParams['figure.figsize'] = (10.0, 5.0)
plt.style.use('ggplot')

In [ ]:
# Some general parameters
n_sample = 5000
rdm_seed = 42
np.random.seed(rdm_seed)

## Probability distributions

There is always a fundamental behaviour beyond collected data. This kind of general rule is called probability distribution (also probability density function (PDF)) of the data. In most use cases - especially when having machine data and technical-scientific relationships - we have a **Normal distribution** (Gaussian distribution). But there are much more [probability distribution](https://en.wikipedia.org/wiki/List_of_probability_distributions). Here we give a short overview on the most important functions which come with [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html).

When performing an experiment that involves randomness, or collecting data from the real world, different outcomes may occur with different probabilities. **Probability distributions** are a way to describe this mathematically: A probability distribution is a mathematical function that describes the probabilities of occurrence of different possible outcomes. When the possible outcomes can take on values in a continuous range (e.g. real numbers), we speak of a **continuous probability distribution**, and typically describe it using a **probability density function (PDF)**, that is, a function that gives the relative likelihood of any possible outcome. 

For example, a **normal distribution** (or **Gaussian distribution**) is very common in technical and scientific data. Below, you see its PDF and how it relates to a **box plot visualization**. Recall how to read a box plot: The boundaries of the box mark the 1st and 3rd quartile (Q1 and Q3), and accordingly, the size of the box shows the inter-quartile range (IQR). Half of all observations fall within this range - that is, 50 percent of the area under the PDF curve.

However, there are many more [probability distributions](https://en.wikipedia.org/wiki/List_of_probability_distributions). Here we give a short overview on the most important functions which come with [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html).


![PDF of normal distribution](https://upload.wikimedia.org/wikipedia/commons/1/1a/Boxplot_vs_PDF.svg)
*Source: [Wikimedia Commons](https://en.wikipedia.org/wiki/File:Boxplot_vs_PDF.svg)*

### Uniform distribution

With a uniform distribution, all results of a random experiment (like throwing a dice, coin tossing, roulette, ...) have the same probability. All possible values have the same frequency.

In [ ]:
model = stats.uniform

y = model.rvs(0, 8, size=n_sample, random_state=rdm_seed)
plt.hist(y, bins=np.linspace(0, 8, 100), color='dodgerblue', density=False, alpha=0.5)
plt.ylabel('Frequency')
plt.title('Uniform distribution');

### Normal distribution (Gaussian distribution)

As mentioned before, a normal distribution is the underlying distribution of many scientific and technical measurements. A fundamental mathematical concept that explains why normal distributions are so common is the [**central limit theorem**](https://en.wikipedia.org/wiki/Central_limit_theorem).

In [ ]:
model = stats.norm

y = model.rvs(0, 1, size=n_sample, random_state=rdm_seed)
plt.hist(y, bins=np.linspace(-4, 4, 100), color='dodgerblue', density=False, alpha=0.5)
plt.ylabel('Frequency')
plt.title('Normal distribution');

### Maxwell–Boltzmann distribution
This Maxwell–Boltzmann distribution describes for example particle speeds in idealized gases.

In [ ]:
model = stats.maxwell

y = model.rvs(0, 1, size=n_sample, random_state=rdm_seed)
plt.hist(y, bins=np.linspace(0, 8, 100), color='dodgerblue', alpha=0.5)
plt.ylabel('Frequency')
plt.title('Maxwell–Boltzmann distribution');

### Exponential distribution
An exponential distribution describes for example the lifetime of atoms having radioactive decay.

In [ ]:
model = stats.expon

y = model.rvs(0, 1, size=n_sample, random_state=rdm_seed)
plt.hist(y, bins=np.linspace(0, 8, 100), color='dodgerblue', alpha=0.5)
plt.ylabel('Frequency')
plt.title('Exponential distribution');

### Normal distribution and fits

If we assume that our data points follow a normal distribution we can fit a Gaussian-PDF to our data. Here we show generated data as a histogram and give the fitted PDF.

In [ ]:
model = stats.norm

# General parameters
n_sample = 100000
n_bins = 51
min_bin = -30
max_bin = 40

bins = np.linspace(min_bin, max_bin, n_bins)
width = (max_bin - min_bin) / (n_bins - 1)
bin_centers = (bins[1:] + bins[:-1]) / 2

# Generate data from a normal distribution
set_mean = 5
set_width = 10
X = bins
y = model.rvs(set_mean, set_width, size=n_sample, random_state=rdm_seed)

# Fit model to data
mean, scale = model.fit(data=y)
print(f'Fitted mean={mean:.3f} and width={scale:.3f}')
y_pdf = model.pdf(bin_centers, loc=mean, scale=scale) * n_sample * width

#Display
norm_data_values, _, _ = plt.hist(y, bins=bins, color='dodgerblue', alpha=0.5, label='Normal distribution')
plt.plot(bin_centers, y_pdf, label='PDF')

plt.ylabel('Frequency')
plt.legend();

### Maximum-Likelihood function
For the previous fit a [Maximum-Liklihood-Fit](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) was used. The Likelihood function is a strong tool for fitting especially when having huge, multidimensional data (e.g. in the case of machine learning). The basic idea is to maximize the likelihood function which can be calculated for each data point instead of finding a full analytical way to estimate all parameters of a model. The total function is the product of all single values. Because of technical reasons the negative logarithmic likelihood function is used to transform it to a minimization task. Here the liklihood function is plotted. The red cross marks the estimated minimum, which is almost the given mean and width of the normal distribution.

In [ ]:
from stats_functions import likelihood
#??likelihood  # Uncomment to show code

In [ ]:
likelihood(model=model, data=y, dim=2, x_limits=(set_width-0.3, set_width+0.3), y_limits=(set_mean-0.4, set_mean+0.4), grid_res=(100, 100), n_ticks=3, x_label='Width', y_label='Mean', x_y_ref_point=(scale, mean))

The same function in two 1D-plots.

In [ ]:
likelihood(model=model, data=y, dim=1, x_limits=(set_width-0.3, set_width+0.3), y_limits=(set_mean-0.4, set_mean+0.4), grid_res=(100, 100), n_ticks=3, x_label='Width', y_label='Mean', x_y_ref_point=(scale, mean))

### Evaluation of the fit
In real world data there is never a perfect fit (see also overfitting later in this notebook). Visual inspection often helps to get a first glance on the quality of a fit. In addition we can use the **residuals** or **standardized residuals** (also called **pulls**) for validation and their distribution. A residual is the distance of a datapoint to the fitted PDF. A histogram of the pulls should result in a normal distribution.

In [ ]:
# Estimate residuals
residuals = norm_data_values - y_pdf
pulls = residuals / np.sqrt(norm_data_values) #standardize residuals

In [ ]:
# Plot pulls
plt.bar(bin_centers, pulls, color='dodgerblue')
plt.title('Standardized residuals or pulls')
plt.ylim((-2.5, 2.5));

In [ ]:
# Plot histrogram of pulls
plt.hist(pulls, color='dodgerblue', bins=15, density=True)
plt.title('Histrogram of pulls')
plt.xlim((-2.5, 2.5));

In [ ]:
from stats_functions import pullpdf
pullpdf(model, (mean, scale), y, nbins=n_bins)

## Methods for fitting

### Example: Least square fit

A [least square fit](https://en.wikipedia.org/wiki/Least_squares) is a standard approach to derive a reasonable fit especially in technical use cases when an obvious functional relation between the input and output parameter is known. The most trivial case is a fit of a line to linearly dependent sample.

The idea is simple: Find the set of parameters of a given function so the sum of the squared **residuals** is minimized.

We use some generated data point. In general they follow a linear relationship of $y = 1 + 3x$ but we will add some random noise.

In [ ]:
# Model or function we want to fit
def linear(x, intercept, slope):
    return intercept + slope * x

In [ ]:
n_sample = 50
err = 1

# Generate example data
x = np.linspace(-2, 2, n_sample)
y = 1 + 3 * x
yerr = np.random.normal(0, err, size=n_sample)
y = y + yerr

# Linear fit
params, cov = opt.curve_fit(linear, x, y, absolute_sigma=err)

# Show results
print(f'intercept={params[0]:.2f}, slope={params[1]:.2f}')
plt.errorbar(x, y, err, marker='+', linewidth=0, elinewidth=0, color='dodgerblue', label='Data points');
plt.plot(x, linear(x, *params), label='Linear fit')
plt.legend();

#### Task

Try out different values for `err`. What happens? What happens when you do not provide the uncertainty of each data point to the fit (remove `absolute_sigma=err`)?

## Overfitting & Underfitting (bias–variance dilemma)

In principle, regardless of the fitting method, the choice of the model will have the highest impact. There are two extremes when chosing a model or function:

- **Underfitting** (bias)
- **Overfitting** (variance)

### Underfitting

Underfitting occurs when our model does not have enough parametric "freedom" to describe the data. In the example we have data coming from a quadratic function while we use a linear function as fit model.

In [ ]:
n_sample = 50
err = 1

# Generate example data (quadratic)
x = np.linspace(-2, 2, n_sample)
y = 2 * x**2
yerr = np.random.normal(0, err, size=n_sample)
y = y + yerr

# Linear fit
params, cov = opt.curve_fit(linear, x, y, absolute_sigma=err)
print(f'intercept={params[0]:.2f}, slope={params[1]:.2f}')

# Show results
plt.errorbar(x, y, err, marker='+', linewidth=0, elinewidth=0, color='dodgerblue', label='Data points');
plt.plot(x, linear(x, *params), label='Linear fit')
plt.legend();

### Correct fitting

Changing to a quadratic model will get us a valid fit.

In [ ]:
def quadratic(x, a, b):
    return a + b * x**2

In [ ]:
# Quadratic fit to the data
params, cov = opt.curve_fit(quadratic, x, y, absolute_sigma=err)

# Show results
print(f'a={params[0]:.2f}, b={params[1]:.2f}')
plt.errorbar(x, y, err, marker='+', linewidth=0, elinewidth=0, color='dodgerblue', label='Data points');
plt.plot(x, quadratic(x, *params), label='Quadratic fit')
plt.legend();

### Overfitting

The opposite problem is called overfitting: The model describes the data points "too well", e.g. it describes random noise instead of simplifying and describing the general relationships in the data. Let's have fewer data points following a linear relationship. We use a linear function and a polynomial function (degree of six) for a regression.

In [ ]:
def poly6(x, a, b, c, d, e, f, g):
    return a + b * x + c * x**2 + d * x**3 + e * x**4 + f * x**5 + g * x**6

In [ ]:
# Generate data
n_sample = 7
err = 3

# Generation of data
x = np.linspace(-2, 2, n_sample)
y = 1 + 3 * x  # Linear function!
yerr = np.random.normal(0, err, size=n_sample)
y = y + yerr

In [ ]:
# Plot data
plt.errorbar(x, y, err, marker='+', linewidth=0, elinewidth=0, color='dodgerblue', label='Data points');

# Linear fit
if False:
    params1, cov1 = opt.curve_fit(linear, x, y, absolute_sigma=err)
    plt.plot(x, linear(x, *params1), label='Linear fit')

# Polynomial fit
if True:
    params2, cov2 = opt.curve_fit(poly6, x, y, absolute_sigma=err)
    x_res = np.linspace(-2.2, 2.15, n_sample * 100)
    plt.plot(x_res, poly6(x_res, *params2), label='Polynomial fit')

# Add new data points
if False:
    x_new = np.array([-2.5, -1,-0.5,0,1])
    y_new = 1 + 3 * x_new
    yerr_new = np.random.normal(0, err, size=5)
    y_new = y_new + yerr_new
    plt.scatter(x_new, y_new, marker='*', s=150, color='orange', label='New data');

plt.legend();

The polynomial can describe each data point exactly but does not get the overall linear dependency of our original linear relationship. Especially at the edges of the spectrum it even diverges. If we generate new data points belonging to the linear relationship they will not be represented very well. 

---

_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2025 [Point 8 GmbH](https://point-8.de)_